<a href="https://colab.research.google.com/github/Rishwanth11/GEN-AI-UMADESU-SIR-/blob/main/FETCH%20FIRST%205%20MAILS%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 52.7 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.173.0
    Uninstalling google-api-python-client-2.173.0:
      Successfully uninstalled google-api-python-client-2.173.0
ERROR: unknown command "install requests" - maybe you meant "install"


In [2]:
# Upload Gmail credentials.json file securely via Colab uploader
from google.colab import files
uploaded =files.upload()

Saving client_secret_522860287561-batmfcp5fk4j7p667af7kgilkf1s6nes.apps.googleusercontent.com.json to client_secret_522860287561-batmfcp5fk4j7p667af7kgilkf1s6nes.apps.googleusercontent.com.json


In [3]:
#cell3
import os
import pickle
import base64
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

from google_auth_oauthlib.flow import Flow

flow = Flow.from_client_secrets_file(
    'client_secret_522860287561-batmfcp5fk4j7p667af7kgilkf1s6nes.apps.googleusercontent.com.json',
    scopes=SCOPES,
    redirect_uri='urn:ietf:wg:oauth:2.0:oob'
)
auth_url, _ = flow.authorization_url(prompt='consent')

print("Please visit this URL to authorize:")
print(auth_url)

# Paste authorization code
code = input("Enter authorization code: ")
flow.fetch_token(code=code)
creds = flow.credentials


Please visit this URL to authorize:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=522860287561-batmfcp5fk4j7p667af7kgilkf1s6nes.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=k8WYv9TbUVn6L9aCFgRDcYQN1zPNyi&prompt=consent&access_type=offline
Enter authorization code: 4/1AVMBsJiTQ3p31Z3lB8DncTqhs7BFX9SAGGtaBUs1-9GJXDWVSWwcvTao_SE


In [13]:
# Initialize Gmail API service
service = build('gmail', 'v1', credentials=creds)

# Fetch recent 5 emails for processing
results = service.users().messages().list(userId='me', maxResults=5).execute()
messages = results.get('messages', [])
email_bodies = []

for msg in messages:
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()
    payload = txt['payload']
    parts = payload.get("parts")

    email_text = None
    if parts:
        for part in parts:
            if part.get("mimeType") == "text/plain" or part.get("mimeType") == "text/html":
                if part.get("body") and part['body'].get('data'):
                    data = part['body'].get('data')
                    byte_code = base64.urlsafe_b64decode(data)
                    email_text = byte_code.decode("utf-8")
                    break # Found a text part, no need to check further
    elif payload.get("body") and payload['body'].get('data'):
        # Handle cases where the body is directly in the payload (less common for complex emails)
        data = payload['body'].get('data')
        byte_code = base64.urlsafe_b64decode(data)
        email_text = byte_code.decode("utf-8")


    if email_text:
        email_bodies.append(email_text)


print("✅ Successfully fetched", len(email_bodies), "emails for AI analysis.")

✅ Successfully fetched 5 emails for AI analysis.


In [14]:
import requests

# Insert your OpenRouter API key directly
openrouter_api_key = "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"  # Replace with your actual OpenRouter key

# Define your system prompt exactly as before
system_prompt = """
You are an Enterprise Smart Notification Agent for campus automation.

Your task for each email:
1. Generate a 5-line summary.
2. Extract action deadlines and tasks.
3. Classify category: Academic, HR, Finance, IT, General.
4. Assign urgency: Critical, High, Normal.
5. Extract: Action required? Yes/No.

Output strictly in format:
Summary: <summary>
Category: <category>
Urgency: <urgency>
Action: <yes/no>
"""

# Define your Smart Notification Agent function using OpenRouter
def smart_notification_agent_openrouter(email_text):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {openrouter_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "openai/gpt-4o",  # You can switch to claude-3.5-sonnet, mistral, etc
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": email_text}
        ],
        "max_tokens": 512,
        "temperature": 0.3
    }

    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    result = response.json()
    return result['choices'][0]['message']['content']

# Call your function over fetched email_bodies
for idx, email_text in enumerate(email_bodies):
    print(f"\n===== Email {idx+1} =====")
    result = smart_notification_agent_openrouter(email_text)
    print(result)


===== Email 1 =====
Summary: Your Amazon package has been delivered, and you are invited to rate the delivery experience. You can also return or replace items if needed through the provided link. The email includes your order number for reference. Note that replies to this email are not possible as it is a no-reply address. 
Category: General
Urgency: Normal
Action: Yes

===== Email 2 =====
Summary: GADIDESI GANESH from NRI Institute of Technology has sent a LinkedIn connection request to Burla Tulasi Krishna Chaitanya. The email provides links to view common connections, accept the invitation, and view GANESH's profile. It also includes information on why the email was sent and options to unsubscribe from LinkedIn emails. The message is intended to facilitate professional networking on LinkedIn.

Category: General  
Urgency: Normal  
Action: Yes  

===== Email 3 =====
Summary: A new sign-in to your Google Account was detected on a Windows device. If this was you, no action is needed.

In [16]:
import gspread
from google.oauth2.service_account import Credentials

# Your service account file (already uploaded to Colab)
SERVICE_ACCOUNT_FILE = 'serviceaccount.json'  # Replace if you used a different filename

# Define Sheets API scope
SCOPES_SHEETS = ['https://www.googleapis.com/auth/spreadsheets']

# Authenticate using Service Account
sheet_creds = Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES_SHEETS)

client = gspread.authorize(sheet_creds)

# Connect to your Google Sheet
spreadsheet_id = '1sNBRs-G0cFNn192nzivch3LVwRRcavsS-w2P5DglNYo'  # Replace with your actual spreadsheet ID
sheet = client.open_by_key(spreadsheet_id).sheet1

# (Optional) Write header row if not already written
sheet.append_row(["Date", "Sender", "Subject", "Summary", "Category", "Urgency", "ActionRequired"])

{'spreadsheetId': '1sNBRs-G0cFNn192nzivch3LVwRRcavsS-w2P5DglNYo',
 'updates': {'spreadsheetId': '1sNBRs-G0cFNn192nzivch3LVwRRcavsS-w2P5DglNYo',
  'updatedRange': 'Sheet1!A1:G1',
  'updatedRows': 1,
  'updatedColumns': 7,
  'updatedCells': 7}}

In [17]:
import datetime
import time # Import the time module
import requests

# Insert your OpenRouter API key directly
openrouter_api_key = "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"  # Replace with your actual OpenRouter key

# Define your system prompt exactly as before
system_prompt = """
You are an Enterprise Smart Notification Agent for campus automation.

Your task for each email:
1. Generate a 5-line summary.
2. Extract action deadlines and tasks.
3. Classify category: Academic, HR, Finance, IT, General.
4. Assign urgency: Critical, High, Normal.
5. Extract: Action required? Yes/No.

Output strictly in format:
Summary: <summary>
Category: <category>
Urgency: <urgency>
Action: <yes/no>
"""

# Define your Smart Notification Agent function using OpenRouter
def smart_notification_agent_openrouter(email_text):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {openrouter_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "openai/gpt-4o",  # You can switch to claude-3.5-sonnet, mistral, etc
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": email_text}
        ],
        "max_tokens": 512,
        "temperature": 0.3
    }

    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    result = response.json()
    return result['choices'][0]['message']['content']

# Call your function over fetched email_bodies
for idx, email_text in enumerate(email_bodies):
    print(f"\n===== Processing Email {idx+1} =====")

    # Assuming you have fetched email metadata (like sender, subject, date) in the email fetching step
    # Replace with actual metadata from your fetched emails
    # For now, using placeholder values
    date_today = datetime.datetime.now().strftime("%Y-%m-%d") # Using YYYY-MM-DD format
    sender = f"Sender {idx+1}" # Placeholder
    subject = f"Subject {idx+1}" # Placeholder

    try:
        # Call the correct function that uses OpenRouter
        result = smart_notification_agent_openrouter(email_text)
        print(result)

        # Parse the LLM output more robustly
        lines = result.strip().split('\n')
        summary, category, urgency, action = "", "", "", ""

        for line in lines:
            if line.startswith("Summary:"):
                summary = line.split(":", 1)[1].strip()
            elif line.startswith("Category:"):
                category = line.split(":", 1)[1].strip()
            elif line.startswith("Urgency:"):
                urgency = line.split(":", 1)[1].strip()
            elif line.startswith("Action:"):
                action = line.split(":", 1)[1].strip()

        # Insert into Google Sheet
        try:
            sheet.append_row([date_today, sender, subject, summary, category, urgency, action])
            print("✅ Successfully inserted data into Google Sheet.")
        except Exception as e:
            print(f"❌ Error inserting data into Google Sheet: {e}")

    except Exception as e:
        print(f"❌ Error processing email {idx+1}: {e}")

    # Add a small delay to avoid hitting rate limits
    time.sleep(1) # Sleep for 1 second between API calls


===== Processing Email 1 =====
Summary: Your Amazon package has been delivered, and you are invited to rate your delivery experience. Links are provided for feedback and for managing your orders, including returns or replacements. The email is sent from a non-reply address. 
Category: General
Urgency: Normal
Action: Yes
✅ Successfully inserted data into Google Sheet.

===== Processing Email 2 =====
Summary: GADIDESI GANESH from NRI Institute of Technology has sent a LinkedIn invitation to Burla Tulasi Krishna Chaitanya. The email includes links to accept the invitation and view GADIDESI's profile. It also provides options to manage LinkedIn email settings and unsubscribe. The invitation is part of LinkedIn's networking features, allowing users to connect with others in their professional network. This email is intended for Burla, who attended the same institute as GADIDESI.
Category: General
Urgency: Normal
Action: Yes
✅ Successfully inserted data into Google Sheet.

===== Processing 